In [1]:
# mypy: disable-error-code="no-untyped-def, no-untyped-call"
# pyright: reportUnusedExpression=false, reportDuplicateImport=false, reportUnusedImport=false
import os
from pathlib import Path

p = Path.cwd()
for _ in range(100):
    if p.name == "cmomy":
        break
    p = p.parent

os.environ["NUMBA_CACHE_DIR"] = str(p / ".numba_cache")

In [2]:
%%html
<style>
    div.output_stderr {
    display: none;
    }
</style>


In [3]:
import numpy as np

np.set_printoptions(precision=4)

# Usage

The basic usage of cmomy is to manipulate central moments.  
We measure two quantities pretend quantities.  The 'energy' and the 'position' of a thing.  We'll construct the average value for each record. Lets say 100 samples each.  

In [4]:
import numpy as np
import xarray as xr

import cmomy

rng = cmomy.random.default_rng(seed=0)

nsamp = 100
energy = xr.DataArray(rng.random(nsamp), dims=["samp"])

position = xr.DataArray(rng.random((nsamp, 3)), dims=["samp", "dim"])

# weight associated with each sample and each record
w = xr.DataArray(rng.random(nsamp), dims=["samp"])

# average over the samples
ce = cmomy.CentralMomentsData.from_vals(energy, weight=w, dim="samp", mom=3)
cp = cmomy.CentralMomentsData.from_vals(position, weight=w, dim="samp", mom=3)

In [5]:
print("energy")
ce

energy


<CentralMomentsData(mom_ndim=1)>
<xarray.DataArray (mom_0: 4)> Size: 32B
array([ 5.3105e+01,  5.5314e-01,  8.6378e-02, -5.7464e-03])
Dimensions without coordinates: mom_0

In [6]:
print("position")
cp

position


<CentralMomentsData(mom_ndim=1)>
<xarray.DataArray (dim: 3, mom_0: 4)> Size: 96B
array([[ 5.3105e+01,  5.2437e-01,  6.9091e-02, -5.5768e-04],
       [ 5.3105e+01,  4.9062e-01,  9.5523e-02,  2.2967e-03],
       [ 5.3105e+01,  5.6372e-01,  8.7545e-02, -8.3042e-03]])
Dimensions without coordinates: dim, mom_0

Here, we've used the constructor {meth}`cmomy.CentralMomentsData.from_vals`.  There are a host of other constructors to create {class}`cmomy.CentralMomentsData` and {class}`cmomy.CentralMomentsArray` objects.  Take a look at the docs for further info.

## Dataset

We can also wrap {class}`~xarray.Dataset` objects.  This has some limitations, as the structure is more complicated.  In particular, methods that depend on a fixed array shape, like {meth}`~cmomy.CentralMomentsData.shape`, etc, are not available for wrapped {class}`~xarray.Dataset`.

In [7]:
ds = xr.Dataset({"energy": energy, "position": position})
ctot = cmomy.CentralMomentsData.from_vals(ds, weight=w, dim="samp", mom=3)
ctot

<CentralMomentsData(mom_ndim=1)>
<xarray.Dataset> Size: 128B
Dimensions:   (mom_0: 4, dim: 3)
Dimensions without coordinates: mom_0, dim
Data variables:
    energy    (mom_0) float64 32B 53.11 0.5531 0.08638 -0.005746
    position  (dim, mom_0) float64 96B 53.11 0.5244 ... 0.08754 -0.008304

In [8]:
# verify that this is the same result as the DataArray version above...
xr.testing.assert_equal(ctot["energy"].obj, ce.obj)
xr.testing.assert_equal(ctot["position"].obj, cp.obj)

## Chunked data

You can also use ``dask`` chunked data (assuming [``dask``](https://docs.dask.org/en/stable/) is installed).  For example:

In [9]:
ctot_chunk = cmomy.CentralMomentsData.from_vals(
    ds.chunk({"dim": -1}), weight=w, dim="samp", mom=3
)
ctot_chunk

<CentralMomentsData(mom_ndim=1)>
<xarray.Dataset> Size: 128B
Dimensions:   (mom_0: 4, dim: 3)
Dimensions without coordinates: mom_0, dim
Data variables:
    energy    (mom_0) float64 32B dask.array<chunksize=(4,), meta=np.ndarray>
    position  (dim, mom_0) float64 96B dask.array<chunksize=(3, 4), meta=np.ndarray>

To compute the final result, use {meth}`~cmomy.CentralMomentsData.compute`:

In [10]:
ctot_chunk.compute()

<CentralMomentsData(mom_ndim=1)>
<xarray.Dataset> Size: 128B
Dimensions:   (mom_0: 4, dim: 3)
Dimensions without coordinates: mom_0, dim
Data variables:
    energy    (mom_0) float64 32B 53.11 0.5531 0.08638 -0.005746
    position  (dim, mom_0) float64 96B 53.11 0.5244 ... 0.08754 -0.008304

In [11]:
xr.testing.assert_equal(ctot.obj, ctot_chunk.obj)

## Basic attributes

Notice that there are three `shape` parameters associated with a {class}`~cmomy.CentralMomentsData` object:
 
* {attr}`~cmomy.CentralMomentsData.mom_shape` : shape of the moments.  For single variable, tuple (mom+1,).  For comoments, (mom_0+1, mom_1+1)
* {attr}`~cmomy.CentralMomentsData.val_shape`: shape of the 'values' part of the data.  For scalar data, `val_shape = ()`.  For vector data, this is the shape of the passed observation data.
* {attr}`~cmomy.CentralMomentsData.shape`: total shape of wrapped moments `shape = val_shape + mom_shape`

In [12]:
for name, c in {"energy": ce, "position": cp}.items():
    print(
        f"""
{name} shapes:
    mom_shape: {c.mom_shape}
    val_shape: {c.val_shape}
    shape    : {c.shape}
"""
    )


energy shapes:
    mom_shape: (4,)
    val_shape: ()
    shape    : (4,)


position shapes:
    mom_shape: (4,)
    val_shape: (3,)
    shape    : (3, 4)



To access the underlying data, use the {meth}`cmomy.CentralMomentsData.to_numpy` method.  The structure is:

* `values[...,0]`: total weight
* `values[...,1]`: average/mean value
* `values[...,k>1]`: `kth` central moment.

In [13]:
ce.to_numpy()

array([ 5.3105e+01,  5.5314e-01,  8.6378e-02, -5.7464e-03])

To access all the central moments (zeroth and first included), use the {meth}`~cmomy.CentralMomentsData.cmom` method.

In [14]:
ce.cmom()

<xarray.DataArray (mom_0: 4)> Size: 32B
array([ 1.    ,  0.    ,  0.0864, -0.0057])
Dimensions without coordinates: mom_0

Likewise, the central moments can be converted to raw moments using the {meth}`~cmomy.CentralMomentsData.rmom` method.  This uses the {func}`~cmomy.convert.moments_type` function behind the scenes.  

In [15]:
# <x**k>
ce.rmom()

<xarray.DataArray (mom_0: 4)> Size: 32B
array([1.    , 0.5531, 0.3923, 0.3068])
Dimensions without coordinates: mom_0

Additionally, there are {class}`xarray.DataArray` like attributes

In [16]:
ce.coords

Coordinates:
    *empty*

In [17]:
ce.attrs

{}

In [18]:
ce.sizes

Frozen({'mom_0': 4})

In [19]:
ce.dims

('mom_0',)

To access the underlying data use {meth}`~cmomy.CentralMomentsData.obj` attribute to access the {class}`~xarray.DataArray` style data, or {meth}`~cmomy.CentralMomentsData.to_numpy` method to access the underlying {class}`~numpy.ndarray`.

In [20]:
ce.obj

<xarray.DataArray (mom_0: 4)> Size: 32B
array([ 5.3105e+01,  5.5314e-01,  8.6378e-02, -5.7464e-03])
Dimensions without coordinates: mom_0

In [21]:
ce.to_numpy()

array([ 5.3105e+01,  5.5314e-01,  8.6378e-02, -5.7464e-03])

## Manipulating (co)moments

So we have our averages.  Cool.  Not very special.  But what if instead we repeat our experiment.  Let's say we did the experiment 10 times each time giving a single record.   Then our data would look like

In [22]:
nsamp = 100
nrec = 10
energy = xr.DataArray(rng.random((nrec, nsamp)), dims=["rec", "samp"])
position = xr.DataArray(rng.random((nrec, nsamp, 3)), dims=["rec", "samp", "dim"])

# weight associated with each sample and each record
w = xr.DataArray(rng.random((nrec, nsamp)), dims=["rec", "samp"])

# average over the samples
ce = cmomy.CentralMomentsData.from_vals(energy, weight=w, dim="samp", mom=3)
cp = cmomy.CentralMomentsData.from_vals(position, weight=w, dim="samp", mom=3)

Consider just the energy.  We suspect that there is some correlation between the experiments (they where done in rapid succession).  So we'd like to consider two adjacent experiments as one experiment.  For this, we can use the {meth}`~cmomy.CentralMomentsData.reduce` method with the `block` parameter.

In [23]:
ce

<CentralMomentsData(mom_ndim=1)>
<xarray.DataArray (rec: 10, mom_0: 4)> Size: 320B
array([[ 5.3064e+01,  4.7292e-01,  8.0077e-02,  6.5155e-03],
       [ 4.6744e+01,  4.7915e-01,  7.8450e-02,  1.8555e-03],
       [ 5.3254e+01,  5.0665e-01,  9.3529e-02,  2.1183e-03],
       [ 5.0303e+01,  5.3773e-01,  6.9736e-02, -6.0640e-03],
       [ 4.8231e+01,  5.2371e-01,  7.8316e-02, -3.9247e-03],
       [ 5.1988e+01,  4.4939e-01,  8.1112e-02,  3.2211e-03],
       [ 4.6908e+01,  4.9788e-01,  9.3582e-02, -4.0934e-03],
       [ 5.4791e+01,  4.8149e-01,  8.2545e-02,  8.0189e-04],
       [ 5.3189e+01,  4.6463e-01,  9.5055e-02,  3.3487e-03],
       [ 5.1267e+01,  5.4169e-01,  9.3554e-02, -4.9880e-03]])
Dimensions without coordinates: rec, mom_0

In [24]:
ce.reduce(block=2, dim="rec")

<CentralMomentsData(mom_ndim=1)>
<xarray.DataArray (rec: 5, mom_0: 4)> Size: 160B
array([[ 9.9808e+01,  4.7583e-01,  7.9325e-02,  4.3255e-03],
       [ 1.0356e+02,  5.2174e-01,  8.2213e-02, -2.4102e-03],
       [ 1.0022e+02,  4.8515e-01,  8.1146e-02, -3.6968e-04],
       [ 1.0170e+02,  4.8905e-01,  8.7702e-02, -1.3211e-03],
       [ 1.0446e+02,  5.0245e-01,  9.5802e-02, -8.2761e-04]])
Dimensions without coordinates: rec, mom_0

Instead, we can resample the already averaged data using {meth}`cmomy.CentralMomentsData.resample_and_reduce`.  We produce a 20 new samples taken from the original (averaged) data.

In [25]:
ce_resamp = ce.resample_and_reduce(dim="rec", nrep=20)
ce_resamp

<CentralMomentsData(mom_ndim=1)>
<xarray.DataArray (rep: 20, mom_0: 4)> Size: 640B
array([[ 4.9894e+02,  4.8483e-01,  8.4004e-02,  7.6615e-04],
       [ 5.0316e+02,  5.0209e-01,  8.0869e-02, -1.1553e-04],
       [ 5.1114e+02,  4.8697e-01,  8.6689e-02,  1.2299e-03],
       [ 5.2135e+02,  4.9141e-01,  8.4228e-02, -5.7084e-04],
       [ 5.0592e+02,  4.9809e-01,  8.5436e-02, -8.7446e-04],
       [ 5.1902e+02,  4.9260e-01,  8.5149e-02,  7.5026e-04],
       [ 5.0659e+02,  5.0687e-01,  8.3342e-02, -2.2954e-03],
       [ 5.0490e+02,  4.9669e-01,  8.6755e-02, -8.1764e-04],
       [ 5.1172e+02,  4.9714e-01,  8.1701e-02, -1.5562e-04],
       [ 5.1915e+02,  4.8611e-01,  9.0029e-02,  2.7399e-03],
       [ 5.1257e+02,  4.8789e-01,  8.3948e-02,  1.4324e-03],
       [ 5.0757e+02,  4.9311e-01,  8.1820e-02, -3.7571e-04],
       [ 4.9210e+02,  4.7666e-01,  8.3947e-02,  2.1234e-03],
       [ 5.2529e+02,  4.9417e-01,  8.4350e-02,  1.1154e-03],
       [ 4.9973e+02,  4.9879e-01,  8.9895e-02, -2.9911e-04],
       [ 5.0993e+02,  4.8937e-01,  8.6093e-02,  8.3597e-04],
       [ 5.0860e+02,  4.7908e-01,  8.5023e-02,  9.4369e-04],
       [ 5.0110e+02,  4.9114e-01,  8.4371e-02, -1.0724e-03],
       [ 5.2742e+02,  4.8909e-01,  8.5711e-02,  1.4218e-03],
       [ 4.9154e+02,  4.9741e-01,  8.1855e-02, -3.3717e-04]])
Dimensions without coordinates: rep, mom_0

This is different than the usual 'resample values'.  This is also available if the original data is available.

In [26]:
# consider 'all' the data for this
energy_stack = energy.stack(c=["rec", "samp"])
weight_stack = w.stack(c=["rec", "samp"])

freq = cmomy.resample.random_freq(nrep=20, ndat=energy_stack.sizes["c"])

out = cmomy.CentralMomentsData.from_resample_vals(
    energy.stack(c=["rec", "samp"]),
    weight=w.stack(c=["rec", "samp"]),
    dim="c",
    nrep=20,
    mom=3,
)
out

<CentralMomentsData(mom_ndim=1)>
<xarray.DataArray (rep: 20, mom_0: 4)> Size: 640B
array([[ 5.0942e+02,  5.0895e-01,  8.9066e-02, -3.0147e-03],
       [ 5.1698e+02,  4.9706e-01,  8.5227e-02, -3.9894e-04],
       [ 5.0103e+02,  4.9691e-01,  9.0094e-02, -9.4089e-04],
       [ 5.0963e+02,  4.9565e-01,  8.5977e-02, -1.0900e-03],
       [ 5.1125e+02,  4.9229e-01,  8.7096e-02,  4.5442e-04],
       [ 5.2042e+02,  4.9696e-01,  8.7269e-02,  2.6198e-04],
       [ 5.0418e+02,  4.9766e-01,  8.8148e-02,  3.8173e-04],
       [ 5.0898e+02,  4.9445e-01,  8.6387e-02,  7.6166e-04],
       [ 5.1509e+02,  5.0119e-01,  7.9516e-02, -3.4758e-05],
       [ 5.0554e+02,  4.8843e-01,  8.6996e-02,  8.7707e-04],
       [ 4.9629e+02,  4.9499e-01,  8.4723e-02,  7.9126e-04],
       [ 5.1164e+02,  4.8969e-01,  8.5734e-02,  6.4368e-04],
       [ 5.1084e+02,  5.1009e-01,  8.8193e-02, -1.0546e-03],
       [ 4.9970e+02,  5.0505e-01,  7.8939e-02, -9.1430e-04],
       [ 5.1451e+02,  4.9375e-01,  8.3872e-02,  5.2589e-04],
       [ 5.1432e+02,  4.9317e-01,  8.7871e-02, -1.4375e-03],
       [ 5.1085e+02,  5.2431e-01,  8.5323e-02, -2.4910e-03],
       [ 5.0989e+02,  5.0796e-01,  8.4258e-02, -1.4678e-03],
       [ 5.1412e+02,  4.6816e-01,  8.5636e-02,  3.6142e-03],
       [ 5.2469e+02,  4.9840e-01,  8.4210e-02, -4.8584e-04]])
Dimensions without coordinates: rep, mom_0

We see that the deviation in the moments is similar using the two resampling methods:

In [27]:
out.obj.sel(mom_0=slice(1, None)).std("rep")

<xarray.DataArray (mom_0: 3)> Size: 24B
array([0.0107, 0.0027, 0.0014])
Dimensions without coordinates: mom_0

In [28]:
ce_resamp.obj.sel(mom_0=slice(1, None)).std("rep")

<xarray.DataArray (mom_0: 3)> Size: 24B
array([0.0072, 0.0024, 0.0012])
Dimensions without coordinates: mom_0

We can also reduce our original data across all the records using {meth}`~cmomy.CentralMomentsData.reduce`

In [29]:
ce.reduce(dim="rec")

<CentralMomentsData(mom_ndim=1)>
<xarray.DataArray (mom_0: 4)> Size: 32B
array([ 5.0974e+02,  4.9508e-01,  8.5572e-02, -6.5659e-05])
Dimensions without coordinates: mom_0